In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv('Data/Support_tickets.csv')
data.head()

In [ ]:
columns = ['ticket_id', 'day_of_week_num', 'company_id', 'company_size_cat', 'industry_cat', 'customer_tier_cat',
           'region_cat','past_30d_tickets', 'past_90d_incidents', 'priority_cat', 'customers_affected','downtime_min',
           'error_rate_pct', 'payment_impact_flag', 'security_incident_flag', 'data_loss_flag', 'has_runbook',
           'reported_by_role_cat','booking_channel_cat', 'customer_sentiment_cat', 'description_length']

In [ ]:
filtered_data = data[columns]
filtered_data.head()

In [ ]:
filtered_data.info()

In [ ]:
def percentile_calc(data, column, x):
    return data[column].quantile(x)

In [ ]:
thresholds = {
    'customers_p90': filtered_data['customers_affected'].quantile(0.90),
    'error_rate_p90': filtered_data['error_rate_pct'].quantile(0.90),
    'downtime_p95': filtered_data['downtime_min'].quantile(0.95),
    'downtime_p90': filtered_data['downtime_min'].quantile(0.90),
    'downtime_p75': filtered_data['downtime_min'].quantile(0.75),
}

In [ ]:
print(thresholds['downtime_p95'], thresholds['downtime_p90'], thresholds['downtime_p75'])

In [ ]:
def sla_tier_assignment(data, thresholds):
    if (
        data['priority_cat'] == 3 and
        (
            data['security_incident_flag'] == 1 or
            data['data_loss_flag'] == 1 or
            data['payment_impact_flag'] == 1
        )
    ):
        return 'SLA-0'

    elif (
        data['customer_tier_cat'] == 3 and
        data['downtime_min'] >= thresholds['downtime_p95']
    ):
        return 'SLA-0'

    elif (
        data['priority_cat'] == 3 or
        data['customers_affected'] >= thresholds['customers_p90'] or
        data['error_rate_pct'] >= thresholds['error_rate_p90'] or
        data['customer_tier_cat'] == 3 or
        data['downtime_min'] >= thresholds['downtime_p90']
    ):
        return 'SLA-1'

    elif (
        data['priority_cat'] == 2 or
        data['downtime_min'] >= thresholds['downtime_p75']
    ):
        return 'SLA-2'

    else:
        return 'SLA-3'


In [ ]:
filtered_data['SLA-Tier'] = filtered_data.apply(
    sla_tier_assignment,
    axis=1, thresholds = thresholds)


In [ ]:
def sla_pressure_calc(row):
    if filtered_data['SLA-Tier'] == "SLA-0":
        return (1/1)
    elif filtered_data['SLA-Tier'] == "SLA-1":
        return (1/4)
    elif filtered_data['SLA-Tier'] == "SLA-2":
        return (1/24)
    elif filtered_data['SLA-Tier'] == "SLA-3":
        return (1/72)

In [ ]:
scaler = MinMaxScaler()
filtered_data['n_customers_affected'] = scaler.fit_transform(filtered_data[['customers_affected']])
filtered_data['n_error_rate_pct'] = scaler.fit_transform(filtered_data[['error_rate_pct']])
filtered_data['recent_ticket_score'] = filtered_data['past_30d_tickets'] / max(filtered_data['past_30d_tickets'])
filtered_data['recent_incident_score'] = filtered_data['past_90d_incidents'] / max(filtered_data['past_90d_incidents'])

In [ ]:
def downtime_severiaty(row):
    if row['downtime_min'] <= thresholds['downtime_p75']:
        return 0.3
    elif row['downtime_min'] <=thresholds['downtime_p90'] and row['downtime_min'] >=thresholds['downtime_p75']:
        return 0.2
    elif row['downtime_min'] >=thresholds['downtime_p90']:
        return 0.3

In [ ]:
def incident_flag(row):
    if  row['security_incident_flag'] == 1 or row['data_loss_flag'] == 1 or row['payment_impact_flag'] == 1:
        return 1
    else:
        return 0

In [ ]:
def impact_scorer(row):
    downtime_factor = downtime_severiaty(row)
    incident_factor = incident_flag(row)
    customer_factor = row['n_customers_affected']
    error_factor = row['n_error_rate_pct']

    impact_score = (downtime_factor) + (customer_factor * 0.25) + (error_factor * 0.15) + (incident_factor * 0.3)
    return impact_score


In [ ]:
filtered_data['customer_tier_cat'].unique()

In [ ]:
def customer_weight(row):
    if row['customer_tier_cat'] == 3:
        return 1.5
    elif row['customer_tier_cat'] == 2:
        return 1.2
    elif row['customer_tier_cat'] == 1:
        return 1

In [ ]:
def instability_weight(row):
    instability_factor = (row['recent_ticket_score']*0.6) * (0.4 * row['recent_incident_score'])
    return 1 + (0.5)*instability_factor

In [ ]:
def SLA_Risk_Score(row):
    SLA_risk_score = customer_weight(row) * instability_weight(row) * impact_scorer(row) * sla_pressure_calc(row)
    return SLA_risk_score

In [ ]:
filtered_data['SLA_Risk_Score'] = filtered_data.apply(SLA_Risk_Score, axis=1)